Zaimplementuj aplikację szacującą czas ukończenia półmaratonu dla zadanych danych

1. Umieść dane w Digital Ocean Spaces

1. Napisz notebook, który będzie Twoim pipelinem do trenowania modelu
    * czyta dane z Digital Ocean Spaces
    * czyści je
    * trenuje model (dobierz odpowiednie metryki [feature selection])
    * nowa wersja modelu jest zapisywana lokalnie i do Digital Ocean Spaces

1. Aplikacja
    * opakuj model w aplikację streamlit
    * wdróż (deploy) aplikację za pomocą Digital Ocean AppPlatform 
    * wejściem jest pole tekstowe, w którym użytkownik się przedstawia, mówi o tym
    jaka jest jego płeć, wiek i tempo na 5km
    * jeśli użytkownik podał za mało danych, wyświetl informację o tym jakich danych brakuje
    * za pomocą LLM (OpenAI) wyłuskaj potrzebne dane, potrzebne dla Twojego modelu
    do określenia, do słownika (dictionary lub JSON)
    * tę część podepnij do Langfuse, aby zbierać metryki o skuteczności działania LLM'a



In [1]:
from pycaret.regression import setup, create_model, compare_models, tune_model, finalize_model, predict_model, save_model
import boto3
import os

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Snippet pomocniczy - zmiana czasu na sekundy

import pandas as pd

df = pd.read_csv('halfmarathon_wroclaw_2023__final.csv', sep=';')
df.head()

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
0,1.0,1787,TOMASZ,GRYCKO,NaN,POL,UKS BLIZA WŁADYSŁAWOWO,M,1.0,M30,...,2.926667,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,WROCŁAW,POL,ARKADIUSZGARDZIELEWSKI.PL,M,2.0,M30,...,2.983333,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875
2,3.0,3832,KRZYSZTOF,HADAS,POZNAŃ,POL,NaN,M,3.0,M20,...,3.123333,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475
3,4.0,416,DAMIAN,DYDUCH,KĘPNO,POL,AZS POLITECHNIKA OPOLSKA,M,4.0,M30,...,3.196667,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963
4,5.0,8476,KAMIL,MAŃKOWSKI,MIRKÓW,POL,PARKRUN WROCŁAW,M,5.0,M20,...,3.276667,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654


In [7]:
df['Czas'] = df['Czas'].apply(convert_time_to_seconds)

In [8]:
# 
df['Czas'].head()

0    3899.0
1    3983.0
2    4104.0
3    4216.0
4    4227.0
Name: Czas, dtype: float64

In [ ]:
AWS_ACCESS_KEY_ID=DO00VR487AT3XBGWM6U2
AWS_SECRET_ACCESS_KEY=Aee5dfeMeqFXxLPzhD/y2yXemjI7IG2RoAELQmDTsJI
AWS_ENDPOINT_URL_S3=https://fra1.digitaloceanspaces.com

In [4]:
import boto3
import os

s3 = boto3.client(
    "s3",
    aws_access_key_id="DO00VR487AT3XBGWM6U2",
    aws_secret_access_key="Aee5dfeMeqFXxLPzhD/y2yXemjI7IG2RoAELQmDTsJI",
    endpoint_url="https://fra1.digitaloceanspaces.com"
)

In [40]:
# Ścieżka lokalna do pliku
local_file_path = "halfmarathon_wroclaw_2023__final.csv"

# Nazwa Twojego Space (jak bucket w AWS)
bucket_name = "dominikbucket"

# Klucz (ścieżka + nazwa pliku na Space)
remote_file_key = "dominikbucket/halfmarathon_wroclaw_2023__final.csv"

In [41]:
# Wysyłanie pliku
s3.upload_file(local_file_path, bucket_name, remote_file_key)

print("Plik został pomyślnie przesłany do DigitalOcean Spaces.")

Plik został pomyślnie przesłany do DigitalOcean Spaces.


In [7]:
# Wylistuj pliki w folderze pliki/
response = s3.list_objects_v2(Bucket=bucket_name, Prefix="dominikbucket/")

for obj in response.get('Contents', []):
    print(obj['Key'])

dominikbucket/halfmarathon_wroclaw_2023__final.csv


In [10]:
from io import StringIO
# Pobieranie pliku i ładowanie do DataFrame
response = s3.get_object(Bucket=bucket_name, Key=remote_file_key)
csv_content = response['Body'].read().decode('utf-8')

df = pd.read_csv(StringIO(csv_content), sep=';')

# Sprawdzenie danych
print(df.head())

   Miejsce  Numer startowy       Imię       Nazwisko   Miasto Kraj  \
0      1.0            1787     TOMASZ         GRYCKO      NaN  POL   
1      2.0               3  ARKADIUSZ  GARDZIELEWSKI  WROCŁAW  POL   
2      3.0            3832  KRZYSZTOF          HADAS   POZNAŃ  POL   
3      4.0             416     DAMIAN         DYDUCH    KĘPNO  POL   
4      5.0            8476      KAMIL      MAŃKOWSKI   MIRKÓW  POL   

                     Drużyna Płeć  Płeć Miejsce Kategoria wiekowa  ...  \
0     UKS BLIZA WŁADYSŁAWOWO    M           1.0               M30  ...   
1  ARKADIUSZGARDZIELEWSKI.PL    M           2.0               M30  ...   
2                        NaN    M           3.0               M20  ...   
3   AZS POLITECHNIKA OPOLSKA    M           4.0               M30  ...   
4            PARKRUN WROCŁAW    M           5.0               M20  ...   

   10 km Tempo  15 km Czas 15 km Miejsce Open  15 km Tempo  20 km Czas  \
0     2.926667    00:44:47                1.0     3.106667  

In [11]:
df

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
0,1.0,1787,TOMASZ,GRYCKO,NaN,POL,UKS BLIZA WŁADYSŁAWOWO,M,1.0,M30,...,2.926667,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,WROCŁAW,POL,ARKADIUSZGARDZIELEWSKI.PL,M,2.0,M30,...,2.983333,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875
2,3.0,3832,KRZYSZTOF,HADAS,POZNAŃ,POL,NaN,M,3.0,M20,...,3.123333,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475
3,4.0,416,DAMIAN,DYDUCH,KĘPNO,POL,AZS POLITECHNIKA OPOLSKA,M,4.0,M30,...,3.196667,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963
4,5.0,8476,KAMIL,MAŃKOWSKI,MIRKÓW,POL,PARKRUN WROCŁAW,M,5.0,M20,...,3.276667,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,NaN,8901,KAROL,ŻUKOWSKI,NaN,NaN,Ovb Polska,M,NaN,M30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNF,NaN
8946,NaN,659,KORNELIA,ŻUKROWSKA,NaN,NaN,NaN,K,NaN,K30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNS,NaN
8947,NaN,6402,MAGDALENA,ŻÓRAŃSKA,NaN,NaN,Iławskie Świry Biegowe,K,NaN,K40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNS,NaN
8948,NaN,2771,KATARZYNA,ĆWIĄKALSKA,NaN,NaN,NaN,K,NaN,K40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNS,NaN


In [12]:
missing_rows = df[df['5 km Czas'].isna() & df['5 km Tempo'].isna()]
print(len(missing_rows))

827


In [ ]:
df.info()

In [3]:
df['Wiek'] = 2023 - df['Rocznik']

In [4]:
columns_to_keep = ['Płeć', 'Wiek', '5 km Tempo', 'Czas']

In [5]:
df1 = df[columns_to_keep]
df1

,Płeć,Wiek,5 km Tempo,Czas
0,M,31.0,2.923333,01:04:59
1,M,37.0,2.960000,01:06:23
2,M,27.0,3.153333,01:08:24
3,M,35.0,3.236667,01:10:16
4,M,28.0,3.240000,01:10:27
...,...,...,...,...
8945,M,33.0,NaN,DNF
8946,K,35.0,NaN,DNS
8947,K,42.0,NaN,DNS
8948,K,41.0,NaN,DNS


In [6]:
# Snippet pomocniczy - zmiana czasu na sekundy

def convert_time_to_seconds(time):
    if pd.isnull(time) or time in ['DNS', 'DNF']:
        return None
    time = time.split(':')
    return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])

In [7]:
df1['Czas'] = df1['Czas'].apply(convert_time_to_seconds)

In [8]:
df1.isna().sum()

Płeć            2
Wiek          201
5 km Tempo    827
Czas          800
dtype: int64

In [9]:
df1 = df1.dropna(thresh=3)
df1.isna().sum()

Płeć            0
Wiek          200
5 km Tempo     26
Czas            0
dtype: int64

In [10]:
df1

,Płeć,Wiek,5 km Tempo,Czas
0,M,31.0,2.923333,3899.0
1,M,37.0,2.960000,3983.0
2,M,27.0,3.153333,4104.0
3,M,35.0,3.236667,4216.0
4,M,28.0,3.240000,4227.0
...,...,...,...,...
8145,K,47.0,8.373333,12108.0
8146,K,71.0,9.060000,12116.0
8147,M,71.0,8.156667,12142.0
8148,K,38.0,7.696667,12211.0


In [23]:
missing_rows = df1[df1['Czas'].isna() & df1['5 km Tempo'].isna()]
print(len(missing_rows))

0


In [24]:
exp = setup(
    data=df1,
    target='Czas',
    numeric_features=['Wiek', '5 km Tempo'],
    categorical_features=['Płeć']
)

,Description,Value
0,Session id,689
1,Target,Czas
2,Target type,Regression
3,Original data shape,"(8149, 4)"
4,Transformed data shape,"(8149, 4)"
5,Transformed train set shape,"(5704, 4)"
6,Transformed test set shape,"(2445, 4)"
7,Numeric features,2
8,Categorical features,1
9,Rows with missing values,2.8%


In [25]:
best_model = exp.compare_models(n_select=4)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,299.1150,180728.2111,424.4222,0.8765,0.0539,0.0392,0.0580
lightgbm,Light Gradient Boosting Machine,306.3185,186709.8071,431.6073,0.8724,0.0546,0.0401,0.0750
lasso,Lasso Regression,300.8810,189720.6134,433.1603,0.8705,0.0545,0.0394,0.0110
ridge,Ridge Regression,300.8878,189781.3887,433.2181,0.8705,0.0545,0.0394,0.0130
lar,Least Angle Regression,300.8658,189767.2072,433.1989,0.8705,0.0545,0.0394,0.0120
llar,Lasso Least Angle Regression,300.8810,189720.6299,433.1603,0.8705,0.0545,0.0394,0.0100
br,Bayesian Ridge,300.8828,189781.9873,433.2166,0.8705,0.0545,0.0394,0.0120
lr,Linear Regression,300.8808,189781.7897,433.2156,0.8705,0.0545,0.0394,0.3660
huber,Huber Regressor,296.4494,191638.6447,435.5595,0.8692,0.0547,0.0386,0.0160
rf,Random Forest Regressor,330.5377,215566.7166,463.5834,0.8528,0.0588,0.0434,0.1890


Processing:   0%|          | 0/80 [00:00<?, ?it/s]

In [26]:
model = exp.create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,306.2682,184166.8122,429.1466,0.8781,0.0535,0.0400
1,282.5673,151353.6807,389.0420,0.9006,0.0494,0.0367
2,303.4204,192678.5583,438.9517,0.8641,0.0578,0.0401
3,304.4181,194300.5016,440.7953,0.8712,0.0555,0.0395
4,301.1787,176599.2259,420.2371,0.8812,0.0536,0.0394
5,301.7746,178110.3641,422.0312,0.8844,0.0529,0.0394
6,299.3523,216860.1927,465.6825,0.8492,0.0583,0.0397
7,301.1998,185226.0937,430.3790,0.8682,0.0545,0.0392
8,314.4774,186770.5436,432.1696,0.8813,0.0554,0.0413


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
tuned_model = tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,323.9781,213263.0619,461.8041,0.8588,0.0573,0.0423
1,304.6004,174624.0532,417.8804,0.8854,0.0518,0.0392
2,323.1335,215319.3872,464.0252,0.8481,0.0603,0.0426
3,327.3829,226080.7826,475.4795,0.8501,0.0596,0.0424
4,336.6135,224829.7816,474.1622,0.8487,0.0605,0.0439
5,314.9393,193059.1260,439.3849,0.8747,0.0551,0.0412
6,320.4953,224463.8671,473.7762,0.8439,0.0596,0.0425
7,317.1695,206594.4640,454.5266,0.8530,0.0573,0.0412
8,338.0773,213098.3211,461.6257,0.8646,0.0585,0.0441


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [28]:
best_model = exp.compare_models([model, tuned_model])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Gradient Boosting Regressor,299.1150,180728.2111,424.4222,0.8765,0.0539,0.0392,0.0590
1,Gradient Boosting Regressor,299.1150,180728.2111,424.4222,0.8765,0.0539,0.0392,0.0550


Processing:   0%|          | 0/13 [00:00<?, ?it/s]

In [29]:
best_final_model = exp.finalize_model(model)
best_final_model

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Wiek', '5 km Tempo'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Płeć'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 TransformerWrapper(include=['Płeć'],
                                    transformer=OrdinalEncoder(cols=['Płeć'],
                                                               handle_missing='return_nan',
                                                               mapping=[{'col': 'Płeć',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': K      0
M      1
NaN   -1
dtype: int64}]))),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 GradientBoostingRegressor(random_state=689))])

In [33]:
new_df=pd.DataFrame([{'Płeć':'M', 'Wiek': 20, '5 km Tempo': 5}], index=['0'])
new_df

,Płeć,Wiek,5 km Tempo
0,M,20,5


In [36]:
predictions = predict_model(best_final_model, data=new_df)
predictions

,Płeć,Wiek,5 km Tempo,prediction_label
0,M,20,5,6628.986376


In [37]:
save_model(best_final_model, 'Pipeline_maratończyka')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Wiek', '5 km Tempo'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Płeć'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['Płeć'],
                                     transformer=OrdinalEncoder(cols=['Płeć'],
                                                                handle_missing='return_nan',
                                                                mapping=[{'col': 'Płeć',
                                                                          'data_type': dtype('O'),
                                                                          'mapping': K      0
 M      1
 NaN   -1
 dtype: int64}]))),
         

In [42]:
local_file_path = 'Pipeline_maratończyka.pkl'

remote_file_key = "dominikbucket/Pipeline_maratończyka.pkl"

In [43]:
s3.upload_file(local_file_path, bucket_name, remote_file_key)

print("Plik został pomyślnie przesłany do DigitalOcean Spaces.")

Plik został pomyślnie przesłany do DigitalOcean Spaces.
